In [22]:
import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Convolution2D, Flatten, Dense

2022-10-09 00:07:40.319206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 00:07:41.165647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aarav/Downloads/college/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-09 00:07:41.165704: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-09 00:07:41.304534: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has

In [23]:
data = np.load('final_data.npy')

imgs = data[0]
steering = data[1]

X_train, X_test, y_train, y_test = train_test_split(imgs, steering, test_size = 0.2)

In [24]:
def batch_generator(imgs, steering, batch_size):   
   
    while True:
        
        X = []
        y = []
    
        for i in range(batch_size):
        
            index = np.random.randint(0,len(imgs)-1)
            img = cv.imread(imgs[index])
            
                
            img = cv.cvtColor(img, cv.COLOR_BGR2YUV)
            img = img[60:135,:,:]
            
            img = cv.resize(img , (200,66))
            
            

            if (np.random.rand() > 0.5):

                brighness_cont = np.random.randint(1,4)
                img = img / brighness_cont

            if (np.random.rand() > 0.5):

                img = cv.flip(img, 1)
                steering_angle = -float(steering[index])

            else:
                steering_angle = float(steering[index])
            
            
            X.append(img)
            y.append(steering_angle)
            
        X = np.array(X)
        y = np.array(y)

        yield X,y


In [25]:
model = Sequential()

model.add(Convolution2D(24, (5,5), strides = (2,2), input_shape = (66,200,3), activation = 'elu'))
model.add(Convolution2D(36, (5,5), strides = (2,2), activation = 'elu'))
model.add(Convolution2D(48, (5,5), strides = (2,2), activation = 'elu'))

model.add(Convolution2D(64, (3,3), activation = 'elu'))
model.add(Convolution2D(64, (3,3), activation = 'elu'))

model.add(Flatten())

model.add(Dense(100, activation = 'elu'))
model.add(Dense(50, activation = 'elu'))
model.add(Dense(10, activation = 'elu'))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

2022-10-09 00:08:07.017708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aarav/Downloads/college/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-09 00:08:07.018218: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-09 00:08:07.018327: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aarav-GF63-Thin-9SCXR): /proc/driver/nvidia/version does not exist
2022-10-09 00:08:07.019620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 98, 24)        1824      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 47, 36)        21636     
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 22, 48)         43248     
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 20, 64)         27712     
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 18, 64)         36928     
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 100)               1

In [ ]:
history = model.fit(batch_generator(X_train, y_train, 64), steps_per_epoch = 300, epochs = 8,
         validation_data = batch_generator(X_test,y_test, 64), validation_steps = 200)

In [31]:
model.save('Model.h5')